In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import sklearn as skl
import tensorflow as tf
import yfinance as yf
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm
from pathlib import Path
from sklearn.linear_model import LinearRegression
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import json
import os
import pymongo
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
import seaborn as sb
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

In [2]:
Soybeans=yf.Ticker("ZS=F")
Corn=yf.Ticker("ZC=F")
Wheat=yf.Ticker("ZW=F")
Cotton=yf.Ticker("CT=F")
Crude=yf.Ticker("CL=F")
USDBRL=yf.Ticker("BRL=X")
EURUSD=yf.Ticker("EURUSD=X")
IR=yf.Ticker("^TNX")
Sugar=yf.Ticker("SB=F")
SoyOil=yf.Ticker("ZL=F")

Hold_Signal=-.05
window = 90  # trading days in rolling window
dpy = 252  # trading days per year
ann_factor = dpy / window

In [7]:
Sugar_H=Sugar.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Sugar_H["Range"]=Sugar_H["High"]-Sugar_H["Low"]
Sugar_H=Sugar_H.drop(["High","Low"], axis=1)

Sugar_H['log_rtn'] = np.log(Sugar_H['Close']).diff()
Sugar_H['real_var'] = np.square(Sugar_H['log_rtn']).rolling(window).sum() * ann_factor
Sugar_H['Vol'] = np.sqrt(Sugar_H['real_var'])

Sugar_P=Sugar_H[:len(Sugar_H)-window]
Sugar_P=Sugar_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Sugar_H=Sugar_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Sugar_P=Sugar_P.drop(["Date","Range","Vol"],axis=1)

Sugar_H["Price_Change"]=(Sugar_H["Close"]-Sugar_P["Close"])/Sugar_P["Close"]
Sugar_H["Buy/Hold"]=np.where(Sugar_H["Price_Change"]< Hold_Signal, "Hold","Buy")
Sugar_H=Sugar_H.drop(["Range","Vol"],axis=1).dropna()
Sugar_H

Date  Close  Price_Change Buy/Hold
0    2021-03-25  15.07      0.026567      Buy
1    2021-03-24  15.63      0.050403      Buy
2    2021-03-23  15.43      0.034876      Buy
3    2021-03-22  15.53      0.072514      Buy
4    2021-03-19  15.76      0.075768      Buy
...         ...    ...           ...      ...
5066 2000-11-20   9.76      0.060870      Buy
5067 2000-11-17   9.63      0.044469      Buy
5068 2000-11-16   9.56      0.042530      Buy
5069 2000-11-15   9.10      0.030578      Buy
5070 2000-11-14   8.73      0.008083      Buy

[5071 rows x 4 columns]

In [8]:
SoyOil_H=SoyOil.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
SoyOil_H["Range"]=SoyOil_H["High"]-SoyOil_H["Low"]
SoyOil_H=SoyOil_H.drop(["High","Low"], axis=1)

SoyOil_H['log_rtn'] = np.log(SoyOil_H['Close']).diff()


SoyOil_H['real_var'] = np.square(SoyOil_H['log_rtn']).rolling(window).sum() * ann_factor
SoyOil_H['Vol'] = np.sqrt(SoyOil_H['real_var'])


SoyOil_P=SoyOil_H[:len(SoyOil_H)-window]
SoyOil_P=SoyOil_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
SoyOil_H=SoyOil_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
SoyOil_P=SoyOil_P.drop(["Date","Range","Vol"],axis=1)

SoyOil_H["Price_Change"]=(SoyOil_H["Close"]-SoyOil_P["Close"])/SoyOil_P["Close"]
SoyOil_H["Buy/Hold"]=np.where(SoyOil_H["Price_Change"]< Hold_Signal, "Hold","Buy")
SoyOil_H=SoyOil_H.drop(["Range","Vol"],axis=1).dropna()
SoyOil_H

Date      Close  Price_Change Buy/Hold
0    2021-03-25  55.040001      0.526345      Buy
1    2021-03-24  57.480000      0.620068      Buy
2    2021-03-23  57.020000      0.613469      Buy
3    2021-03-22  56.369999      0.589230      Buy
4    2021-03-19  53.869999      0.566444      Buy
...         ...        ...           ...      ...
4969 2001-01-29  14.500000     -0.066924     Hold
4970 2001-01-26  14.600000     -0.062901     Hold
4971 2001-01-25  14.680000     -0.054124     Hold
4972 2001-01-24  14.580000     -0.062982     Hold
4973 2001-01-23  14.710000     -0.062460     Hold

[4974 rows x 4 columns]

In [28]:
Soybeans_H=Soybeans.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Soybeans_H["Range"]=Soybeans_H["High"]-Soybeans_H["Low"]
Soybeans_H=Soybeans_H.drop(["High","Low"], axis=1)

Soybeans_H['log_rtn'] = np.log(Soybeans_H['Close']).diff()

# Var Swap (returns are not demeaned)
Soybeans_H['real_var'] = np.square(Soybeans_H['log_rtn']).rolling(window).sum() * ann_factor
Soybeans_H['Vol'] = np.sqrt(Soybeans_H['real_var'])


Soybeans_P=Soybeans_H[:len(Soybeans_H)-window]
Soybeans_P=Soybeans_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Soybeans_H=Soybeans_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Soybeans_P=Soybeans_P.drop(["Date","Range","Vol"],axis=1)

Soybeans_H["Price_Change"]=(Soybeans_H["Close"]-Soybeans_P["Close"])/Soybeans_P["Close"]
Soybeans_H["Buy/Hold"]=np.where(Soybeans_H["Price_Change"]< Hold_Signal, "Hold","Buy")
#Soybeans_H["Past_Price"]=Soybeans_P["Close"]
#Soybeans_H=Soybeans_H.set_index('Date')
#Soybeans_P=Soybeans_P.set_index('Date')
#Soybeans_H["Buy/Hold"].hist(figsize = (12,10))
#plt.show()
Soybeans_H=Soybeans_H.drop(["Range","Vol"],axis=1).dropna()
Soybeans_H

Date    Close  Price_Change Buy/Hold
0    2021-03-25  1417.75      0.245552      Buy
1    2021-03-24  1432.75      0.296606      Buy
2    2021-03-23  1423.25      0.295630      Buy
3    2021-03-22  1417.50      0.286590      Buy
4    2021-03-19  1416.25      0.312558      Buy
...         ...      ...           ...      ...
4945 2001-06-13   473.00      0.028820      Buy
4946 2001-06-12   473.00      0.025474      Buy
4947 2001-06-08   458.75     -0.002175      Buy
4948 2001-06-07   461.50     -0.011777      Buy
4949 2001-06-06   452.00     -0.036247      Buy

[4950 rows x 4 columns]

In [11]:
Corn_H=Corn.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Corn_H["Range"]=Corn_H["High"]-Corn_H["Low"]
Corn_H=Corn_H.drop(["High","Low"], axis=1)


Corn_H['log_rtn'] = np.log(Corn_H['Close']).diff()

# Var Swap (returns are not demeaned)
Corn_H['real_var'] = np.square(Corn_H['log_rtn']).rolling(window).sum() * ann_factor
Corn_H['Vol'] = np.sqrt(Corn_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

Corn_P=Corn_H[:len(Corn_H)-90]
Corn_P=Corn_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Corn_H=Corn_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Corn_P=Corn_P.drop(["Date","Range","Vol"],axis=1)

Corn_H["Price_Change"]=(Corn_H["Close"]-Corn_P["Close"])/Corn_P["Close"]
Corn_H["Buy/Hold"]=np.where(Corn_H["Price_Change"]< Hold_Signal, "Hold","Buy")
#Corn_H["Past_Price"]=Corn_P["Close"]
#Corn_H=Corn_H.set_index('Date')
#Corn_P=Corn_P.set_index('Date')
#Corn_H["Buy/Hold"].hist(figsize = (12,10))
#plt.show()
Corn_H=Corn_H.drop(["Range","Vol"],axis=1).dropna()
Corn_H

Date   Close  Price_Change Buy/Hold
0    2021-03-25  548.00      0.295508      Buy
1    2021-03-24  553.25      0.357669      Buy
2    2021-03-23  551.25      0.355255      Buy
3    2021-03-22  549.00      0.341478      Buy
4    2021-03-19  557.75      0.376311      Buy
...         ...     ...           ...      ...
4953 2001-05-23  192.50     -0.125000     Hold
4954 2001-05-22  194.00     -0.110092     Hold
4955 2001-05-21  198.50     -0.078886     Hold
4956 2001-05-18  201.50     -0.040476      Buy
4957 2001-05-17  201.00     -0.038278      Buy

[4958 rows x 4 columns]

In [12]:
Wheat_H=Wheat.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Wheat_H["Range"]=Wheat_H["High"]-Wheat_H["Low"]
Wheat_H=Wheat_H.drop(["High","Low"], axis=1)


Wheat_H['log_rtn'] = np.log(Wheat_H['Close']).diff()

# Var Swap (returns are not demeaned)
Wheat_H['real_var'] = np.square(Wheat_H['log_rtn']).rolling(window).sum() * ann_factor
Wheat_H['Vol'] = np.sqrt(Wheat_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

Wheat_P=Wheat_H[:len(Wheat_H)-90]
Wheat_P=Wheat_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Wheat_H=Wheat_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Wheat_P=Wheat_P.drop(["Date","Range","Vol"],axis=1)

Wheat_H["Price_Change"]=(Wheat_H["Close"]-Wheat_P["Close"])/Wheat_P["Close"]
Wheat_H["Buy/Hold"]=np.where(Wheat_H["Price_Change"]< Hold_Signal, "Hold","Buy")
#Wheat_H["Past_Price"]=Wheat_P["Close"]
#Wheat_H=Wheat_H.set_index('Date')
#Wheat_P=Wheat_P.set_index('Date')
Wheat_H=Wheat_H.drop(["Range","Vol"],axis=1).dropna()
Wheat_H

Date   Close  Price_Change Buy/Hold
0    2021-03-25  612.25      0.006163      Buy
1    2021-03-24  624.75      0.045607      Buy
2    2021-03-23  634.75      0.054402      Buy
3    2021-03-22  627.25      0.029545      Buy
4    2021-03-19  627.00      0.034653      Buy
...         ...     ...           ...      ...
4941 2001-05-04  263.00      0.041584      Buy
4942 2001-05-03  268.25      0.046829      Buy
4943 2001-05-02  271.00      0.056530      Buy
4944 2001-05-01  275.50      0.055556      Buy
4945 2001-04-30  265.75      0.021134      Buy

[4946 rows x 4 columns]

In [13]:
Cotton_H=Cotton.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Cotton_H["Range"]=Cotton_H["High"]-Cotton_H["Low"]
Cotton_H=Cotton_H.drop(["High","Low"], axis=1)


Cotton_H['log_rtn'] = np.log(Cotton_H['Close']).diff()

# Var Swap (returns are not demeaned)
Cotton_H['real_var'] = np.square(Cotton_H['log_rtn']).rolling(window).sum() * ann_factor
Cotton_H['Vol'] = np.sqrt(Cotton_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

Cotton_P=Cotton_H[:len(Cotton_H)-90]
Cotton_P=Cotton_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Cotton_H=Cotton_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Cotton_P=Cotton_P.drop(["Date","Range","Vol"],axis=1)


Cotton_H["Price_Change"]=(Cotton_H["Close"]-Cotton_P["Close"])/Cotton_P["Close"]
Cotton_H["Buy/Hold"]=np.where(Cotton_H["Price_Change"]< Hold_Signal, "Hold","Buy")
#Cotton_H["Past_Price"]=Cotton_P["Close"]
#Cotton_H=Cotton_H.set_index('Date')
#Cotton_P=Cotton_P.set_index('Date')
Cotton_H=Cotton_H.drop(["Range","Vol"],axis=1).dropna()
Cotton_H

Date      Close  Price_Change Buy/Hold
0    2021-03-25  78.440002      0.122978      Buy
1    2021-03-24  82.440002      0.177714      Buy
2    2021-03-23  83.529999      0.217284      Buy
3    2021-03-22  84.620003      0.207650      Buy
4    2021-03-19  84.680000      0.205752      Buy
...         ...        ...           ...      ...
5105 2000-09-26  63.549999      0.023844      Buy
5106 2000-09-25  63.880001      0.020121      Buy
5107 2000-09-22  63.450001      0.011155      Buy
5108 2000-09-21  63.900002      0.026836      Buy
5109 2000-09-20  63.139999      0.038487      Buy

[5110 rows x 4 columns]

In [14]:
Crude_H=Crude.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Crude_H["Range"]=Crude_H["High"]-Crude_H["Low"]
Crude_H=Crude_H.drop(["High","Low"], axis=1)


Crude_H['log_rtn'] = np.log(Crude_H['Close']).diff()

# Var Swap (returns are not demeaned)
Crude_H['real_var'] = np.square(Crude_H['log_rtn']).rolling(window).sum() * ann_factor
Crude_H['Vol'] = np.sqrt(Crude_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

Crude_P=Crude_H[:len(Crude_H)-90]
Crude_P=Crude_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Crude_H=Crude_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Crude_P=Crude_P.drop(["Date","Range","Vol"],axis=1)
Crude_P

Close
0     41.360001
1     40.290001
2     37.139999
3     38.790001
4     39.150002
...         ...
4861  27.719999
4862  27.350000
4863  28.000000
4864  28.200001
4865  27.950001

[4866 rows x 1 columns]

In [15]:
USDBRL_H=USDBRL.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)

USDBRL_H["Range"]=USDBRL_H["High"]-USDBRL_H["Low"]
USDBRL_H=USDBRL_H.drop(["High","Low"], axis=1)


USDBRL_H['log_rtn'] = np.log(USDBRL_H['Close']).diff()

# Var Swap (returns are not demeaned)
USDBRL_H['real_var'] = np.square(USDBRL_H['log_rtn']).rolling(window).sum() * ann_factor
USDBRL_H['Vol'] = np.sqrt(USDBRL_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

USDBRL_P=USDBRL_H[:len(USDBRL_H)-90]
USDBRL_P=USDBRL_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
USDBRL_H=USDBRL_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()#[:len(Soybeans_H)]
USDBRL_P=USDBRL_P.drop(["Date","Range","Vol"],axis=1)
USDBRL_P

Close
0     5.3617
1     5.3272
2     5.4149
3     5.4570
4     5.4567
...      ...
3867  2.8840
3868  2.8840
3869  2.8700
3870  2.8740
3871  2.8750

[3872 rows x 1 columns]

In [16]:
EURUSD_H=EURUSD.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
EURUSD_H["Range"]=EURUSD_H["High"]-EURUSD_H["Low"]
EURUSD_H=EURUSD_H.drop(["High","Low"], axis=1)


EURUSD_H['log_rtn'] = np.log(EURUSD_H['Close']).diff()

# Var Swap (returns are not demeaned)
EURUSD_H['real_var'] = np.square(EURUSD_H['log_rtn']).rolling(window).sum() * ann_factor
EURUSD_H['Vol'] = np.sqrt(EURUSD_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

EURUSD_P=EURUSD_H[:len(EURUSD_H)-90]
EURUSD_P=EURUSD_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
EURUSD_H=EURUSD_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()#[:len(Soybeans_H)]
EURUSD_P=EURUSD_P.drop(["Date","Range","Vol"],axis=1)

EURUSD_P

Close
0     1.184764
1     1.186366
2     1.185944
3     1.184525
4     1.180339
...        ...
4285  1.208605
4286  1.207700
4287  1.216604
4288  1.213106
4289  1.201302

[4290 rows x 1 columns]

In [17]:
IR_H=IR.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
IR_H["Range"]=IR_H["High"]-IR_H["Low"]
IR_H=IR_H.drop(["High","Low"], axis=1)


IR_H['log_rtn'] = np.log(IR_H['Close']).diff()

# Var Swap (returns are not demeaned)
IR_H['real_var'] = np.square(IR_H['log_rtn']).rolling(window).sum() * ann_factor
IR_H['Vol'] = np.sqrt(IR_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

IR_P=IR_H[:len(IR_H)-90]
IR_P=IR_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
IR_H=IR_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()#[:len(Soybeans_H)]
IR_P=IR_P.drop(["Date","Range","Vol"],axis=1)
IR_P

Close
0      0.972
1      0.958
2      0.820
3      0.776
4      0.768
...      ...
14598  3.870
14599  3.860
14600  3.870
14601  3.850
14602  3.820

[14603 rows x 1 columns]

In [18]:
Sugar_H=Sugar_H.rename(columns={"Range": "Sugar Range", "Close": "Sugar Close", "Vol" : "Sugar Vol"})[:len(USDBRL_H)]
SoyOil_H=SoyOil_H.rename(columns={"Range": "SoyOil Range", "Close": "SoyOil Close", "Vol" : "SoyOil Vol"})[:len(USDBRL_H)]
Sugar_P=Sugar_P.rename(columns={"Range": "Sugar P_Range", "Close": "Sugar P_Close", "Vol" : "Sugar P_Vol"})[:len(USDBRL_P)]
SoyOil_P=SoyOil_P.rename(columns={"Range": "SoyOil P_Range", "Close": "SoyOil P_Close", "Vol" : "SoyOil P_Vol"})[:len(USDBRL_P)]

In [22]:
SoyOil_H

Date  SoyOil Close  Price_Change Buy/Hold
0    2021-03-25     55.040001      0.526345      Buy
1    2021-03-24     57.480000      0.620068      Buy
2    2021-03-23     57.020000      0.613469      Buy
3    2021-03-22     56.369999      0.589230      Buy
4    2021-03-19     53.869999      0.566444      Buy
...         ...           ...           ...      ...
3957 2005-05-19     22.040001      0.095427      Buy
3958 2005-05-18     22.260000      0.098175      Buy
3959 2005-05-17     22.020000      0.096614      Buy
3960 2005-05-16     22.160000      0.111334      Buy
3961 2005-05-13     21.980000      0.106747      Buy

[3962 rows x 4 columns]

In [29]:
Soybeans_H=Soybeans_H.rename(columns={"Range": "Soybeans Range", "Close": "Soybeans Close", "Vol" : "Soybeans Vol"})[:len(USDBRL_H)]
Corn_H=Corn_H.rename(columns={"Range": "Corn Range", "Close": "Corn Close", "Vol" : "Corn Vol"})[:len(USDBRL_H)]

Wheat_H=Wheat_H.rename(columns={"Range": "Wheat Range", "Close": "Wheat Close", "Vol" : "Wheat Vol"})[:len(USDBRL_H)]
Cotton_H=Cotton_H.rename(columns={"Range": "Cotton Range","Close": "Cotton Close", "Vol" : "Cotton Vol"})[:len(USDBRL_H)]
Crude_H=Crude_H.rename(columns={"Range": "Crude Range","Close": "Crude Close", "Vol" : "Crude Vol"})[:len(USDBRL_H)]
USDBRL_H=USDBRL_H.rename(columns={"Range": "USDBRL Range", "Close": "USDBRL Close", "Vol" : "USDBRL Vol"})[:len(USDBRL_H)]
EURUSD_H=EURUSD_H.rename(columns={"Range": "EURUSD Range", "Close": "EURUSD Close", "Vol" : "EURUSD Vol"})[:len(USDBRL_H)]
IR_H=IR_H.rename(columns={"Range": "IR Range", "Close": "IR Close", "Vol" : "IR Vol"})[:len(USDBRL_H)]

In [30]:
Soybeans_P=Soybeans_P.rename(columns={"Range": "Soybeans P_Range", "Close": "Soybeans P_Close", "Vol" : "Soybeans P_Vol"})[:len(USDBRL_P)]
Corn_P=Corn_P.rename(columns={"Range": "Corn P_Range", "Close": "Corn P_Close", "Vol" : "Corn P_Vol"})[:len(USDBRL_P)]

Wheat_P=Wheat_P.rename(columns={"Range": "Wheat P_Range", "Close": "Wheat P_Close", "Vol" : "Wheat P_Vol"})[:len(USDBRL_P)]
Cotton_P=Cotton_P.rename(columns={"Range": "Cotton P_Range","Close": "Cotton P_Close", "Vol" : "Cotton P_Vol"})[:len(USDBRL_P)]
Crude_P=Crude_P.rename(columns={"Range": "Crude P_Range","Close": "Crude P_Close", "Vol" : "Crude P_Vol"})[:len(USDBRL_P)]
USDBRL_P=USDBRL_P.rename(columns={"Range": "USDBRL P_Range", "Close": "USDBRL P_Close", "Vol" : "USDBRL P_Vol"})[:len(USDBRL_P)]
EURUSD_P=EURUSD_P.rename(columns={"Range": "EURUSD P_Range", "Close": "EURUSD P_Close", "Vol" : "EURUSD P_Vol"})[:len(USDBRL_P)]
IR_P=IR_P.rename(columns={"Range": "IR P_Range", "Close": "IR P_Close", "Vol" : "IR P_Vol"})[:len(USDBRL_P)]

In [31]:
Prices_S = [Corn_P, Wheat_P, Cotton_P, Crude_P, USDBRL_P, EURUSD_P, IR_P, SoyOil_P, Sugar_P]
Prices_C= [Soybeans_P, Wheat_P, Cotton_P, Crude_P, USDBRL_P, EURUSD_P, IR_P, SoyOil_P, Sugar_P]
Prices_W= [Soybeans_P, Corn_P, Cotton_P, Crude_P, USDBRL_P, EURUSD_P, IR_P,SoyOil_P, Sugar_P]
Prices_Ctn= [Soybeans_P, Wheat_P, Corn_P, Crude_P, USDBRL_P, EURUSD_P, IR_P, SoyOil_P, Sugar_P]
Prices_Sb=[Corn_P, Wheat_P, Cotton_P, Crude_P, USDBRL_P, EURUSD_P, IR_P,Soybeans_P,SoyOil_P]
Prices_SO=[Corn_P, Wheat_P, Cotton_P, Crude_P, USDBRL_P, EURUSD_P, IR_P,Soybeans_P,Sugar_P]

In [32]:
for price in Prices_S:
    Soybeans_H=Soybeans_H.merge(price, how="outer", left_index=True, right_index=True)
    
Soybeans_H=Soybeans_H.drop("Date", axis=1)
Soybeans_H


Soybeans Close  Price_Change Buy/Hold  Corn P_Close  Wheat P_Close  \
0            1417.75      0.245552      Buy        423.00         608.50   
1            1432.75      0.296606      Buy        407.50         597.50   
2            1423.25      0.295630      Buy        406.75         602.00   
3            1417.50      0.286590      Buy        409.25         609.25   
4            1416.25      0.312558      Buy        405.25         606.00   
...              ...           ...      ...           ...            ...   
3957          629.25      0.123661      Buy           NaN            NaN   
3958          633.25      0.147712      Buy           NaN            NaN   
3959          620.75      0.145824      Buy           NaN            NaN   
3960          623.50      0.167603      Buy           NaN            NaN   
3961          603.75      0.139689      Buy           NaN            NaN   

      Cotton P_Close  Crude P_Close  USDBRL P_Close  EURUSD P_Close  \
0          69.849998      41.360001          5.3617        1.184764   
1          70.000000      40.290001          5.3272        1.186366   
2          68.620003      37.139999          5.4149        1.185944   
3          70.070000      38.790001          5.4570        1.184525   
4          70.230003      39.150002          5.4567        1.180339   
...              ...            ...             ...             ...   
3957             NaN            NaN             NaN             NaN   
3958             NaN            NaN             NaN             NaN   
3959             NaN            NaN             NaN             NaN   
3960             NaN            NaN             NaN             NaN   
3961             NaN            NaN             NaN             NaN   

      IR P_Close  SoyOil P_Close  Sugar P_Close  
0          0.972       36.060001          14.68  
1          0.958       35.480000          14.88  
2          0.820       35.340000          14.91  
3          0.776       35.470001          14.48  
4          0.768       34.389999          14.65  
...          ...             ...            ...  
3957         NaN             NaN            NaN  
3958         NaN             NaN            NaN  
3959         NaN             NaN            NaN  
3960         NaN             NaN            NaN  
3961         NaN             NaN            NaN  

[3962 rows x 12 columns]

In [33]:
for price in Prices_C:
    Corn_H=Corn_H.merge(price, how="outer", left_index=True, right_index=True)
    
Corn_H=Corn_H.drop("Date", axis=1)
Corn_H

Corn Close  Price_Change Buy/Hold  Soybeans P_Close  Wheat P_Close  \
0         548.00      0.295508      Buy           1138.25         608.50   
1         553.25      0.357669      Buy           1105.00         597.50   
2         551.25      0.355255      Buy           1098.50         602.00   
3         549.00      0.341478      Buy           1101.75         609.25   
4         557.75      0.376311      Buy           1079.00         606.00   
...          ...           ...      ...               ...            ...   
3957      207.50     -0.005988      Buy               NaN            NaN   
3958      207.00      0.027295      Buy               NaN            NaN   
3959      195.25     -0.028607      Buy               NaN            NaN   
3960      196.25     -0.027261      Buy               NaN            NaN   
3961      198.25     -0.031746      Buy               NaN            NaN   

      Cotton P_Close  Crude P_Close  USDBRL P_Close  EURUSD P_Close  \
0          69.849998      41.360001          5.3617        1.184764   
1          70.000000      40.290001          5.3272        1.186366   
2          68.620003      37.139999          5.4149        1.185944   
3          70.070000      38.790001          5.4570        1.184525   
4          70.230003      39.150002          5.4567        1.180339   
...              ...            ...             ...             ...   
3957             NaN            NaN             NaN             NaN   
3958             NaN            NaN             NaN             NaN   
3959             NaN            NaN             NaN             NaN   
3960             NaN            NaN             NaN             NaN   
3961             NaN            NaN             NaN             NaN   

      IR P_Close  SoyOil P_Close  Sugar P_Close  
0          0.972       36.060001          14.68  
1          0.958       35.480000          14.88  
2          0.820       35.340000          14.91  
3          0.776       35.470001          14.48  
4          0.768       34.389999          14.65  
...          ...             ...            ...  
3957         NaN             NaN            NaN  
3958         NaN             NaN            NaN  
3959         NaN             NaN            NaN  
3960         NaN             NaN            NaN  
3961         NaN             NaN            NaN  

[3962 rows x 12 columns]

In [34]:
for price in Prices_W:
    Wheat_H=Wheat_H.merge(price, how="outer", left_index=True, right_index=True)
    
Wheat_H=Wheat_H.drop("Date", axis=1)
Wheat_H

Wheat Close  Price_Change Buy/Hold  Soybeans P_Close  Corn P_Close  \
0          612.25      0.006163      Buy           1138.25        423.00   
1          624.75      0.045607      Buy           1105.00        407.50   
2          634.75      0.054402      Buy           1098.50        406.75   
3          627.25      0.029545      Buy           1101.75        409.25   
4          627.00      0.034653      Buy           1079.00        405.25   
...           ...           ...      ...               ...           ...   
3957       309.50      0.005686      Buy               NaN           NaN   
3958       315.00      0.016949      Buy               NaN           NaN   
3959       310.00      0.005677      Buy               NaN           NaN   
3960       306.25      0.021685      Buy               NaN           NaN   
3961       296.50      0.001689      Buy               NaN           NaN   

      Cotton P_Close  Crude P_Close  USDBRL P_Close  EURUSD P_Close  \
0          69.849998      41.360001          5.3617        1.184764   
1          70.000000      40.290001          5.3272        1.186366   
2          68.620003      37.139999          5.4149        1.185944   
3          70.070000      38.790001          5.4570        1.184525   
4          70.230003      39.150002          5.4567        1.180339   
...              ...            ...             ...             ...   
3957             NaN            NaN             NaN             NaN   
3958             NaN            NaN             NaN             NaN   
3959             NaN            NaN             NaN             NaN   
3960             NaN            NaN             NaN             NaN   
3961             NaN            NaN             NaN             NaN   

      IR P_Close  SoyOil P_Close  Sugar P_Close  
0          0.972       36.060001          14.68  
1          0.958       35.480000          14.88  
2          0.820       35.340000          14.91  
3          0.776       35.470001          14.48  
4          0.768       34.389999          14.65  
...          ...             ...            ...  
3957         NaN             NaN            NaN  
3958         NaN             NaN            NaN  
3959         NaN             NaN            NaN  
3960         NaN             NaN            NaN  
3961         NaN             NaN            NaN  

[3962 rows x 12 columns]

In [35]:
for price in Prices_Ctn:
    Cotton_H=Cotton_H.merge(price, how="outer", left_index=True, right_index=True)

Cotton_H=Cotton_H.drop("Date", axis=1)    
Cotton_H

Cotton Close  Price_Change Buy/Hold  Soybeans P_Close  Wheat P_Close  \
0        78.440002      0.122978      Buy           1138.25         608.50   
1        82.440002      0.177714      Buy           1105.00         597.50   
2        83.529999      0.217284      Buy           1098.50         602.00   
3        84.620003      0.207650      Buy           1101.75         609.25   
4        84.680000      0.205752      Buy           1079.00         606.00   
...            ...           ...      ...               ...            ...   
3957     53.750000      0.208137      Buy               NaN            NaN   
3958     55.080002      0.248413      Buy               NaN            NaN   
3959     56.799999      0.268707      Buy               NaN            NaN   
3960     55.930000      0.270848      Buy               NaN            NaN   
3961     55.009998      0.244570      Buy               NaN            NaN   

      Corn P_Close  Crude P_Close  USDBRL P_Close  EURUSD P_Close  IR P_Close  \
0           423.00      41.360001          5.3617        1.184764       0.972   
1           407.50      40.290001          5.3272        1.186366       0.958   
2           406.75      37.139999          5.4149        1.185944       0.820   
3           409.25      38.790001          5.4570        1.184525       0.776   
4           405.25      39.150002          5.4567        1.180339       0.768   
...            ...            ...             ...             ...         ...   
3957           NaN            NaN             NaN             NaN         NaN   
3958           NaN            NaN             NaN             NaN         NaN   
3959           NaN            NaN             NaN             NaN         NaN   
3960           NaN            NaN             NaN             NaN         NaN   
3961           NaN            NaN             NaN             NaN         NaN   

      SoyOil P_Close  Sugar P_Close  
0          36.060001          14.68  
1          35.480000          14.88  
2          35.340000          14.91  
3          35.470001          14.48  
4          34.389999          14.65  
...              ...            ...  
3957             NaN            NaN  
3958             NaN            NaN  
3959             NaN            NaN  
3960             NaN            NaN  
3961             NaN            NaN  

[3962 rows x 12 columns]

In [36]:
for price in Prices_Sb:
    Sugar_H=Sugar_H.merge(price, how="outer", left_index=True, right_index=True)

Sugar_H=Sugar_H.drop("Date", axis=1)    
Sugar_H

Sugar Close  Price_Change Buy/Hold  Corn P_Close  Wheat P_Close  \
0           15.07      0.026567      Buy        423.00         608.50   
1           15.63      0.050403      Buy        407.50         597.50   
2           15.43      0.034876      Buy        406.75         602.00   
3           15.53      0.072514      Buy        409.25         609.25   
4           15.76      0.075768      Buy        405.25         606.00   
...           ...           ...      ...           ...            ...   
3957         8.35     -0.075304     Hold           NaN            NaN   
3958         8.26     -0.099237     Hold           NaN            NaN   
3959         8.49     -0.060841     Hold           NaN            NaN   
3960         8.39     -0.073951     Hold           NaN            NaN   
3961         8.36     -0.073171     Hold           NaN            NaN   

      Cotton P_Close  Crude P_Close  USDBRL P_Close  EURUSD P_Close  \
0          69.849998      41.360001          5.3617        1.184764   
1          70.000000      40.290001          5.3272        1.186366   
2          68.620003      37.139999          5.4149        1.185944   
3          70.070000      38.790001          5.4570        1.184525   
4          70.230003      39.150002          5.4567        1.180339   
...              ...            ...             ...             ...   
3957             NaN            NaN             NaN             NaN   
3958             NaN            NaN             NaN             NaN   
3959             NaN            NaN             NaN             NaN   
3960             NaN            NaN             NaN             NaN   
3961             NaN            NaN             NaN             NaN   

      IR P_Close  Soybeans P_Close  SoyOil P_Close  
0          0.972           1138.25       36.060001  
1          0.958           1105.00       35.480000  
2          0.820           1098.50       35.340000  
3          0.776           1101.75       35.470001  
4          0.768           1079.00       34.389999  
...          ...               ...             ...  
3957         NaN               NaN             NaN  
3958         NaN               NaN             NaN  
3959         NaN               NaN             NaN  
3960         NaN               NaN             NaN  
3961         NaN               NaN             NaN  

[3962 rows x 12 columns]

In [37]:
for price in Prices_SO:
    SoyOil_H=SoyOil_H.merge(price, how="outer", left_index=True, right_index=True)
    
SoyOil_H=SoyOil_H.drop("Date", axis=1)
SoyOil_H

SoyOil Close  Price_Change Buy/Hold  Corn P_Close  Wheat P_Close  \
0        55.040001      0.526345      Buy        423.00         608.50   
1        57.480000      0.620068      Buy        407.50         597.50   
2        57.020000      0.613469      Buy        406.75         602.00   
3        56.369999      0.589230      Buy        409.25         609.25   
4        53.869999      0.566444      Buy        405.25         606.00   
...            ...           ...      ...           ...            ...   
3957     22.040001      0.095427      Buy           NaN            NaN   
3958     22.260000      0.098175      Buy           NaN            NaN   
3959     22.020000      0.096614      Buy           NaN            NaN   
3960     22.160000      0.111334      Buy           NaN            NaN   
3961     21.980000      0.106747      Buy           NaN            NaN   

      Cotton P_Close  Crude P_Close  USDBRL P_Close  EURUSD P_Close  \
0          69.849998      41.360001          5.3617        1.184764   
1          70.000000      40.290001          5.3272        1.186366   
2          68.620003      37.139999          5.4149        1.185944   
3          70.070000      38.790001          5.4570        1.184525   
4          70.230003      39.150002          5.4567        1.180339   
...              ...            ...             ...             ...   
3957             NaN            NaN             NaN             NaN   
3958             NaN            NaN             NaN             NaN   
3959             NaN            NaN             NaN             NaN   
3960             NaN            NaN             NaN             NaN   
3961             NaN            NaN             NaN             NaN   

      IR P_Close  Soybeans P_Close  Sugar P_Close  
0          0.972           1138.25          14.68  
1          0.958           1105.00          14.88  
2          0.820           1098.50          14.91  
3          0.776           1101.75          14.48  
4          0.768           1079.00          14.65  
...          ...               ...            ...  
3957         NaN               NaN            NaN  
3958         NaN               NaN            NaN  
3959         NaN               NaN            NaN  
3960         NaN               NaN            NaN  
3961         NaN               NaN            NaN  

[3962 rows x 12 columns]

In [44]:
Data=[Corn_H, Wheat_H, Cotton_H, Crude_H, USDBRL_H, EURUSD_H, IR_H,Soybeans_H,Sugar_H, SoyOil_H]
client = MongoClient("mongodb://localhost:27017/Group_Project")
db=client.Group_Project


In [51]:
records=json.loads(Soybeans_H.T.to_json()).values()
db.Soybeans.insert(records)

[ObjectId('60538ba80aa32c20e3a4dc7a'),
 ObjectId('60538ba80aa32c20e3a4dc7b'),
 ObjectId('60538ba80aa32c20e3a4dc7c'),
 ObjectId('60538ba80aa32c20e3a4dc7d'),
 ObjectId('60538ba80aa32c20e3a4dc7e'),
 ObjectId('60538ba80aa32c20e3a4dc7f'),
 ObjectId('60538ba80aa32c20e3a4dc80'),
 ObjectId('60538ba80aa32c20e3a4dc81'),
 ObjectId('60538ba80aa32c20e3a4dc82'),
 ObjectId('60538ba80aa32c20e3a4dc83'),
 ObjectId('60538ba80aa32c20e3a4dc84'),
 ObjectId('60538ba80aa32c20e3a4dc85'),
 ObjectId('60538ba80aa32c20e3a4dc86'),
 ObjectId('60538ba80aa32c20e3a4dc87'),
 ObjectId('60538ba80aa32c20e3a4dc88'),
 ObjectId('60538ba80aa32c20e3a4dc89'),
 ObjectId('60538ba80aa32c20e3a4dc8a'),
 ObjectId('60538ba80aa32c20e3a4dc8b'),
 ObjectId('60538ba80aa32c20e3a4dc8c'),
 ObjectId('60538ba80aa32c20e3a4dc8d'),
 ObjectId('60538ba80aa32c20e3a4dc8e'),
 ObjectId('60538ba80aa32c20e3a4dc8f'),
 ObjectId('60538ba80aa32c20e3a4dc90'),
 ObjectId('60538ba80aa32c20e3a4dc91'),
 ObjectId('60538ba80aa32c20e3a4dc92'),
 ObjectId('60538ba80aa32c

In [52]:
records=json.loads(Sugar_H.T.to_json()).values()
db.Sugar.insert(records)

[ObjectId('60538bd50aa32c20e3a4ebef'),
 ObjectId('60538bd50aa32c20e3a4ebf0'),
 ObjectId('60538bd50aa32c20e3a4ebf1'),
 ObjectId('60538bd50aa32c20e3a4ebf2'),
 ObjectId('60538bd50aa32c20e3a4ebf3'),
 ObjectId('60538bd50aa32c20e3a4ebf4'),
 ObjectId('60538bd50aa32c20e3a4ebf5'),
 ObjectId('60538bd50aa32c20e3a4ebf6'),
 ObjectId('60538bd50aa32c20e3a4ebf7'),
 ObjectId('60538bd50aa32c20e3a4ebf8'),
 ObjectId('60538bd50aa32c20e3a4ebf9'),
 ObjectId('60538bd50aa32c20e3a4ebfa'),
 ObjectId('60538bd50aa32c20e3a4ebfb'),
 ObjectId('60538bd50aa32c20e3a4ebfc'),
 ObjectId('60538bd50aa32c20e3a4ebfd'),
 ObjectId('60538bd50aa32c20e3a4ebfe'),
 ObjectId('60538bd50aa32c20e3a4ebff'),
 ObjectId('60538bd50aa32c20e3a4ec00'),
 ObjectId('60538bd50aa32c20e3a4ec01'),
 ObjectId('60538bd50aa32c20e3a4ec02'),
 ObjectId('60538bd50aa32c20e3a4ec03'),
 ObjectId('60538bd50aa32c20e3a4ec04'),
 ObjectId('60538bd50aa32c20e3a4ec05'),
 ObjectId('60538bd50aa32c20e3a4ec06'),
 ObjectId('60538bd50aa32c20e3a4ec07'),
 ObjectId('60538bd50aa32c

In [53]:
records=json.loads(SoyOil_H.T.to_json()).values()
db.SoyOil.insert(records)

[ObjectId('60538bea0aa32c20e3a4fb64'),
 ObjectId('60538bea0aa32c20e3a4fb65'),
 ObjectId('60538bea0aa32c20e3a4fb66'),
 ObjectId('60538bea0aa32c20e3a4fb67'),
 ObjectId('60538bea0aa32c20e3a4fb68'),
 ObjectId('60538bea0aa32c20e3a4fb69'),
 ObjectId('60538bea0aa32c20e3a4fb6a'),
 ObjectId('60538bea0aa32c20e3a4fb6b'),
 ObjectId('60538bea0aa32c20e3a4fb6c'),
 ObjectId('60538bea0aa32c20e3a4fb6d'),
 ObjectId('60538bea0aa32c20e3a4fb6e'),
 ObjectId('60538bea0aa32c20e3a4fb6f'),
 ObjectId('60538bea0aa32c20e3a4fb70'),
 ObjectId('60538bea0aa32c20e3a4fb71'),
 ObjectId('60538bea0aa32c20e3a4fb72'),
 ObjectId('60538bea0aa32c20e3a4fb73'),
 ObjectId('60538bea0aa32c20e3a4fb74'),
 ObjectId('60538bea0aa32c20e3a4fb75'),
 ObjectId('60538bea0aa32c20e3a4fb76'),
 ObjectId('60538bea0aa32c20e3a4fb77'),
 ObjectId('60538bea0aa32c20e3a4fb78'),
 ObjectId('60538bea0aa32c20e3a4fb79'),
 ObjectId('60538bea0aa32c20e3a4fb7a'),
 ObjectId('60538bea0aa32c20e3a4fb7b'),
 ObjectId('60538bea0aa32c20e3a4fb7c'),
 ObjectId('60538bea0aa32c

In [54]:
records=json.loads(Corn_H.T.to_json()).values()
db.Corn.insert(records)

[ObjectId('60538bfd0aa32c20e3a50ad9'),
 ObjectId('60538bfd0aa32c20e3a50ada'),
 ObjectId('60538bfd0aa32c20e3a50adb'),
 ObjectId('60538bfd0aa32c20e3a50adc'),
 ObjectId('60538bfd0aa32c20e3a50add'),
 ObjectId('60538bfd0aa32c20e3a50ade'),
 ObjectId('60538bfd0aa32c20e3a50adf'),
 ObjectId('60538bfd0aa32c20e3a50ae0'),
 ObjectId('60538bfd0aa32c20e3a50ae1'),
 ObjectId('60538bfd0aa32c20e3a50ae2'),
 ObjectId('60538bfd0aa32c20e3a50ae3'),
 ObjectId('60538bfd0aa32c20e3a50ae4'),
 ObjectId('60538bfd0aa32c20e3a50ae5'),
 ObjectId('60538bfd0aa32c20e3a50ae6'),
 ObjectId('60538bfd0aa32c20e3a50ae7'),
 ObjectId('60538bfd0aa32c20e3a50ae8'),
 ObjectId('60538bfd0aa32c20e3a50ae9'),
 ObjectId('60538bfd0aa32c20e3a50aea'),
 ObjectId('60538bfd0aa32c20e3a50aeb'),
 ObjectId('60538bfd0aa32c20e3a50aec'),
 ObjectId('60538bfd0aa32c20e3a50aed'),
 ObjectId('60538bfd0aa32c20e3a50aee'),
 ObjectId('60538bfd0aa32c20e3a50aef'),
 ObjectId('60538bfd0aa32c20e3a50af0'),
 ObjectId('60538bfd0aa32c20e3a50af1'),
 ObjectId('60538bfd0aa32c

In [55]:
records=json.loads(Wheat_H.T.to_json()).values()
db.Wheat.insert(records)

[ObjectId('60538c140aa32c20e3a51a4e'),
 ObjectId('60538c140aa32c20e3a51a4f'),
 ObjectId('60538c140aa32c20e3a51a50'),
 ObjectId('60538c140aa32c20e3a51a51'),
 ObjectId('60538c140aa32c20e3a51a52'),
 ObjectId('60538c140aa32c20e3a51a53'),
 ObjectId('60538c140aa32c20e3a51a54'),
 ObjectId('60538c140aa32c20e3a51a55'),
 ObjectId('60538c140aa32c20e3a51a56'),
 ObjectId('60538c140aa32c20e3a51a57'),
 ObjectId('60538c140aa32c20e3a51a58'),
 ObjectId('60538c140aa32c20e3a51a59'),
 ObjectId('60538c140aa32c20e3a51a5a'),
 ObjectId('60538c140aa32c20e3a51a5b'),
 ObjectId('60538c140aa32c20e3a51a5c'),
 ObjectId('60538c140aa32c20e3a51a5d'),
 ObjectId('60538c140aa32c20e3a51a5e'),
 ObjectId('60538c140aa32c20e3a51a5f'),
 ObjectId('60538c140aa32c20e3a51a60'),
 ObjectId('60538c140aa32c20e3a51a61'),
 ObjectId('60538c140aa32c20e3a51a62'),
 ObjectId('60538c140aa32c20e3a51a63'),
 ObjectId('60538c140aa32c20e3a51a64'),
 ObjectId('60538c140aa32c20e3a51a65'),
 ObjectId('60538c140aa32c20e3a51a66'),
 ObjectId('60538c140aa32c

In [56]:
records=json.loads(Cotton_H.T.to_json()).values()
db.Cotton.insert(records)

[ObjectId('60538c260aa32c20e3a529c3'),
 ObjectId('60538c260aa32c20e3a529c4'),
 ObjectId('60538c260aa32c20e3a529c5'),
 ObjectId('60538c260aa32c20e3a529c6'),
 ObjectId('60538c260aa32c20e3a529c7'),
 ObjectId('60538c260aa32c20e3a529c8'),
 ObjectId('60538c260aa32c20e3a529c9'),
 ObjectId('60538c260aa32c20e3a529ca'),
 ObjectId('60538c260aa32c20e3a529cb'),
 ObjectId('60538c260aa32c20e3a529cc'),
 ObjectId('60538c260aa32c20e3a529cd'),
 ObjectId('60538c260aa32c20e3a529ce'),
 ObjectId('60538c260aa32c20e3a529cf'),
 ObjectId('60538c260aa32c20e3a529d0'),
 ObjectId('60538c260aa32c20e3a529d1'),
 ObjectId('60538c260aa32c20e3a529d2'),
 ObjectId('60538c260aa32c20e3a529d3'),
 ObjectId('60538c260aa32c20e3a529d4'),
 ObjectId('60538c260aa32c20e3a529d5'),
 ObjectId('60538c260aa32c20e3a529d6'),
 ObjectId('60538c260aa32c20e3a529d7'),
 ObjectId('60538c260aa32c20e3a529d8'),
 ObjectId('60538c260aa32c20e3a529d9'),
 ObjectId('60538c260aa32c20e3a529da'),
 ObjectId('60538c260aa32c20e3a529db'),
 ObjectId('60538c260aa32c